In [1]:
using DataFrames, JSON, BenchmarkTools, InlineStrings

In [2]:
json_source = "/Users/dbuchaca/Datasets/amazon_product_data/Home_and_Kitchen_5.json"
asins = [String15(x["asin"]) for x in JSON.parse.(readlines(open(json_source)))]

# Make the dataset bigger for benchmarking purposes
for i in 1:9
    append!(asins, asins)
end

In [6]:
length(asins)

282461184

In [9]:
json_source = "/Users/dbuchaca/Datasets/amazon_product_data/Video_Games_5.json"
blocklist = [x["asin"] for x in JSON.parse.(readlines(open(json_source)))];
length(blocklist)

231780

In [4]:
using InlineStrings

function check_list_in_blocklist2(list, blocklist)
    blocklist_set = Set{InlineString15}(blocklist)
    result = Set{InlineString15}([])
    for l in list
        if l in blocklist_set
            push!(result,l)
        end
    end
    return result
end

check_list_in_blocklist2 (generic function with 1 method)

In [10]:
asins_s15 = InlineString15.(asins)
asins2_s15 = InlineString15.(blocklist);

@btime check_list_in_blocklist2(asins_s15,asins2_s15)

  4.211 s (13 allocations: 8.50 MiB)


Set{String15}()

In [11]:
@code_llvm asins_s15[1] == asins_s15[2]

;  @ /Users/dbuchaca/.julia/packages/InlineStrings/rlLZO/src/InlineStrings.jl:340 within `==`
define i8 @"julia_==_2024"(i128 zeroext %0, i128 zeroext %1) #0 {
top:
  %2 = icmp eq i128 %0, %1
  %3 = zext i1 %2 to i8
  ret i8 %3
}


In [12]:
# Note that this hardware will deal with 2 Xors for the 2 sets of 64 bits
@code_native asins_s15[1] == asins_s15[2]

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 12, 0
	.globl	"_julia_==_2061"                ## -- Begin function julia_==_2061
	.p2align	4, 0x90
"_julia_==_2061":                       ## @"julia_==_2061"
; ┌ @ /Users/dbuchaca/.julia/packages/InlineStrings/rlLZO/src/InlineStrings.jl:340 within `==`
	.cfi_startproc
## %bb.0:                               ## %top
	xorq	%rcx, %rsi
	xorq	%rdx, %rdi
	orq	%rsi, %rdi
	sete	%al
	retq
	.cfi_endproc
; └
                                        ## -- End function
.subsections_via_symbols


In [ ]:
json_source = "/Users/dbuchaca/Datasets/amazon_product_data/Home_and_Kitchen_5.json"
asins = [String15(x["asin"]) for x in JSON.parse.(readlines(open(json_source)))]

json_source = "/Users/dbuchaca/Datasets/amazon_product_data/Video_Games_5.json"
asins_2 = [x["asin"] for x in JSON.parse.(readlines(open(json_source)))]

In [ ]:
@btime intersect(Set(asins_2), Set(asins))

In [ ]:
@btime asins_2_set = Set(asins_2);

In [ ]:
@btime asins_2_set = Set(asins_2);

In [ ]:
function check_list_in_blocklist(list, blocklist)
    blocklist_set = Set{String}(blocklist)
    result = Set([])
    for l in list
        if l in blocklist_set
            push!(result,l)
        end
    end
    return result
end

In [ ]:
@btime check_list_in_blocklist(asins,asins_2)

In [ ]:
using InlineStrings

function check_list_in_blocklist2(list, blocklist)
    blocklist_set = Set{InlineString15}(blocklist)
    result = Set{InlineString15}([])
    for l in list
        if l in blocklist_set
            push!(result,l)
        end
    end
    return result
end

In [ ]:
asins_s15 = InlineString15.(asins)
asins2_s15 = InlineString15.(asins_2);

@btime check_list_in_blocklist2(asins_s15,asins2_s15)

In [ ]:
asins_s15

In [ ]:
function check_list_in_blocklist2(list, blocklist)
    blocklist_set = Set{String}(blocklist)
    return intersect(Set{String}(list),blocklist_set )
end

In [ ]:
@btime check_list_in_blocklist2($asins, $asins_2)

In [ ]:
function get_chars(list_of_strings)
    unique_chars = Dict{Char, Int}([])
    for s in list_of_strings
        for c in s
            if haskey(unique_chars, c) == false
                unique_chars[c] = length(unique_chars) +1
            end
        end
    end
    return unique_chars
end

In [ ]:
symbol_to_int = get_chars(asins)

In [ ]:
sort([x for x in values(symbol_to_int)])

### Embedding each possible asin to an integer

In [ ]:
function hash_value(str, symbol_to_int)
    result = 0
    n_symbols = length(symbol_to_int)
    for (i,c) in enumerate(str)
        result += symbol_to_int[c]^i
    end
    return result
end

In [ ]:
@btime hash_value("1111111111", symbol_to_int)

In [ ]:
max_hash = hash_value("HHHHHHHHHH", symbol_to_int)

In [ ]:
max_hash < typemax(Int64)

In [ ]:
hash_value("1111111111", symbol_to_int)

In [ ]:
@btime max_hash == asins[10]

How much faster could we do comparison between strings and Ints?

In [ ]:
a1 = asins[1]
a2 = asins[2]
@btime a1 == a2

In [ ]:
ha1 = hash_value(asins[1],symbol_to_int)
ha2 = hash_value(asins[2],symbol_to_int)
@btime ha1 == ha2

Note that the cost of converting the strings to ints is already higher than the comparison cost, making this irrelevant unless data is already stored with hashes

In [ ]:
function hash_value_ascii(str)
    result = 0
    for (i,c) in enumerate(str)
        result += Int(c)^i
        println(i, "-->", c, ' ', result)
    end
    return result
end

In [ ]:
hash_value_ascii(asins[4])

In [ ]:
function hash_value_ascii(s)
    result = 0
    for (i,c) in enumerate(s)
        result += Int(c)^i
    end
    return result
end

In [ ]:
a = asins[100]
@btime hash_value_ascii($a) 

In [ ]:
function hash_value_ascii(s)

    a1 =  Int(s[1])
    a2 =  Int(s[2])^1
    a3 =  Int(s[3])^2
    a4 =  Int(s[4])^3
    a5 =  Int(s[5])^4
    a6 =  Int(s[6])^5
    a7 =  Int(s[7])^6
    a8 =  Int(s[8])^7
    a9 =  Int(s[9])^8
    a10 =  Int(s[10])^9

    return a1+a2+a3+a4+a5+a6+a7+a8+a9+a10
end

In [ ]:
a = "474829382az"
@btime hash_value_ascii($a)

In [ ]:
aux = Int8.(codeunits(a))

In [ ]:
@btime aux == aux

In [ ]:
a1 = ascii("HHHHHHHHHH")

In [ ]:
b"HHHHHHHHHH"

In [ ]:
a1 = asins[1]
a2 = asins[1000]

@btime aux = codeunits($a1) == codeunits($a2) 

In [ ]:
a1 = asins[1]
a2 = asins[1000]

function check_equal_str(a1,a2)
    return a1 == a2
end

@btime check_equal_str($a1,$a2)
@btime check_equal_str(a1,a2)

In [ ]:
a1 = asins[1]
a2 = asins[1000]

function check_equal_codeunits(a1,a2)
    return codeunits(a1) == codeunits(a2) 
end

@btime check_equal_codeunits($a1,$a2)
@btime check_equal_codeunits(a1,a2)

In [ ]:

sum([127^i for i in 0:9])

In [ ]:
sum([127^i for i in 0:9])<typemax(Int64)

In [ ]:
function hash_value_ascii(str)
    result = 0
    for (i,c) in enumerate(str)
        result += Int(c)^i
        println(i, "-->", c, ' ', result)
    end
    return result
end